## **Installs**

In [ ]:
! pip install torch==2.0.1 transformers datasets peft accelerate trl bitsandbytes optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━

## **Import**

In [ ]:
import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, GPTQConfig GenerationConfig
from trl import DPOTrainer
import time

## **HuggingFace Login**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## **Params**

In [ ]:
HG_MODEL_NAME = "TheBloke/OpenHermes-2-Mistral-7B-GPTQ"
HG_TOKENIZER_NAME = HG_MODEL_NAME
HG_DATASET_NAME = "HuggingFaceH4/ultrafeedback_binarized"
#TOKEN = 'ENTER TOKEN HERE'

## **Dataset and Preprocessing**

In [ ]:
def hg_data(hg_dataset_name, split, token):
    dataset = load_dataset(
        hg_dataset_name,
        split = split,
        token = token
    )

    original_columns = dataset.column_names

    dataset = dataset.map(
        lambda sample: {
          "prompt": [prompt for prompt in sample["prompt"]],
          "chosen": sample["chosen"],
          "rejected": sample["rejected"],
        },
        batched=True,
        remove_columns=original_columns,
    )

    train_df = dataset.to_pandas().dropna()

    train_df["chosen"] = train_df["chosen"].str.get(1).str.get("content")
    train_df["rejected"] = train_df["rejected"].str.get(1).str.get("content")

    val_df = train_df.sample(10)

    train_data = Dataset.from_pandas(train_df)
    val_data = Dataset.from_pandas(val_df)

    return train_data, val_data

In [ ]:
  train_data, val_data = hg_data(HG_DATASET_NAME, "train_prefs", TOKEN)

Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## **Models and Tokenizer**

In [ ]:
model = AutoModelForCausalLM.from_pretrained(HG_MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=GPTQConfig(bits=4, disable_exllama=True))

model_ref = AutoModelForCausalLM.from_pretrained(HG_MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=GPTQConfig(bits=4, disable_exllama=True))

tokenizer = AutoTokenizer.from_pretrained(HG_TOKENIZER_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## **Peft Config and Model Setup**

In [ ]:
peft_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["q_proj", "v_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
peft_config.inference_mode = False

In [ ]:
model = prepare_model_for_kbit_training(model)
model.config.use_cache=False
model.gradient_checkpointing_enable()
model.config.pretraining_tp=1
model = get_peft_model(model, peft_config)

## **Training**

In [ ]:
training_args = TrainingArguments(
        per_device_train_batch_size=1,
        max_steps=50,
        remove_unused_columns=False,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        evaluation_strategy="steps",
        logging_first_step=True,
        logging_steps=10,
        output_dir="openhermes-mistral-dpo-gptq",
        optim="paged_adamw_32bit",
        warmup_steps=2,
        fp16=True,
        push_to_hub=True
    )

In [ ]:
dpo_trainer = DPOTrainer(
        model,
        model_ref,
        args=training_args,
        beta=0.1,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        max_length=512,
        max_target_length=256,
        max_prompt_length=256
    )

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
10,0.683400,0.664873,0.015108,-0.061587,0.562500,0.076695,-167.887161,-138.869629,-2.672401,-2.571630
20,0.721900,0.648841,-0.002720,-0.105784,0.562500,0.103064,-168.329147,-139.047913,-2.674421,-2.578634
30,0.629100,0.628809,-0.006313,-0.141868,0.562500,0.135555,-168.689972,-139.083847,-2.670776,-2.576525
40,0.657700,0.627371,-0.038313,-0.175188,0.562500,0.136875,-169.023193,-139.403839,-2.667314,-2.571048
50,0.691000,0.626274,-0.041946,-0.185611,0.562500,0.143665,-169.127411,-139.440170,-2.665607,-2.568697


TrainOutput(global_step=50, training_loss=0.6768183326721191, metrics={'train_runtime': 359.2137, 'train_samples_per_second': 0.139, 'train_steps_per_second': 0.139, 'total_flos': 0.0, 'train_loss': 0.6768183326721191, 'epoch': 0.03})

In [ ]:
dpo_trainer.save_model("/models")

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1707517278.d347c2f7288a.192.0:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.16k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

## **Inference**

In [ ]:
input = tokenizer("""I have dropped my phone in water. Now it is not working what should I do now?""", return_tensors="pt").to("cuda")

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    "openhermes-mistral-dpo-gptq",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

st_time = time.time()
trained_output = trained_model.generate(**input, generation_config=generation_config)
print(tokenizer.decode(trained_output[0], skip_special_tokens=True))
print(time.time()-st_time)


st_time = time.time()
ref_output = model_ref.generate(**input, generation_config=generation_config)
print(tokenizer.decode(ref_output[0], skip_special_tokens=True))
print(time.time()-st_time)

I have dropped my phone in water. Now it is not working what should I do now?

If you have dropped your phone in water, the first thing you should do is to turn it off immediately. If it is still on, turn it off. Then remove the battery if possible. If the battery is not removable, then leave the phone off.

Next, you should try to dry the phone as much as possible. You can use a hair dryer or a fan to dry the phone. You can also use uncooked rice or silica gel packets to absorb the moisture.

After the phone has dried, you can try to turn it on. If it does not turn on, you can try to charge it. If it still does not turn on, then you may need to take it to a professional for repair.

If you have dropped your phone in water and it is not working, you should try to dry it as soon as possible. If it still does not work after drying, you may need to take it to a professional for repair.
0.0028336048126220703
I have dropped my phone in water. Now it is not working what should I do now?

If 